In [14]:
import pandas as pd
import numpy as np
import nmslib # Для KNN
import torch
import os
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from tqdm.auto import tqdm # Для отображения прогресса 
from collections import Counter 
import pandas as pd
import numpy as np

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [15]:
TRAIN_DATA_PATH = 'dataset_train.tsv'
TEST_DATA_PATH = 'dataset_test.tsv'

print(f"Загрузка обучающих данных из: {TRAIN_DATA_PATH}")
print(f"Загрузка тестовых данных из: {TEST_DATA_PATH}")

try:
    train_df = pd.read_csv(TRAIN_DATA_PATH, sep='\t', header=None, names=['text', 'intent'], quoting=3)
    test_df = pd.read_csv(TEST_DATA_PATH, sep='\t', header=None, names=['text', 'intent'], quoting=3)

    text_col = 'text'
    label_col = 'intent'

    print("\nДанные успешно загружены.")
    print("Пример обучающих данных:")
    print(train_df.head())
    print("\nРаспределение по классам (Train):")
    print(train_df[label_col].value_counts())
    print(f"\nВсего классов: {train_df[label_col].nunique()}")

    train_texts = train_df[text_col].astype(str).tolist() 
    train_labels = train_df[label_col].tolist()
    test_texts = test_df[text_col].astype(str).tolist()
    test_labels = test_df[label_col].tolist()

    print(f"\nРазмер обучающей выборки: {len(train_texts)}")
    print(f"Размер тестовой выборки: {len(test_texts)}")

except FileNotFoundError:
    print(f"Ошибка: Файлы не найдены. Убедитесь, что {TRAIN_DATA_PATH} и {TEST_DATA_PATH} находятся в папке E:\\cv_project")
    exit()
except Exception as e:
    print(f"Произошла ошибка при чтении или обработке файлов: {e}")
    exit()


Загрузка обучающих данных из: dataset_train.tsv
Загрузка тестовых данных из: dataset_test.tsv

Данные успешно загружены.
Пример обучающих данных:
                         text             intent
0           мне нужна справка  statement_general
1            оформить справку  statement_general
2               взять справку  statement_general
3        справку как получить  statement_general
4  справку ммф где получаться  statement_general

Распределение по классам (Train):
intent
sched_teacher                1110
sched_for_group               405
sched_for_group_day           402
wifi                          287
status_free                   271
                             ... 
smalltalk_talk                 23
student_trade_union_enter      22
nsu_foundation_date            22
location_general               20
staff_trade_union_enter        20
Name: count, Length: 142, dtype: int64

Всего классов: 142

Размер обучающей выборки: 13230
Размер тестовой выборки: 883


In [16]:
# MODEL_NAME = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1'
MODEL_NAME = 'sberbank-ai/sbert_large_nlu_ru' # Попробуйте эту, если первая не устроит

print(f"\nЗагрузка модели: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)#

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) 
print(f"Модель будет использовать: {device}")


Загрузка модели: sberbank-ai/sbert_large_nlu_ru
Модель будет использовать: cpu


In [17]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# Функция для генерации эмбеддингов для списка текстов (с батчингом)
def get_embeddings(texts, batch_size=64): # Увеличила batch_size для скорости
    all_embeddings = []
    model.eval() 
    # Используем tqdm для отображения прогресс-бара
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating Embeddings"):
        batch_texts = texts[i:i+batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        all_embeddings.append(batch_embeddings.cpu().numpy())
        
    return np.vstack(all_embeddings)

In [18]:
print("\nГенерация эмбеддингов для обучающей выборки...")
train_embeddings = get_embeddings(train_texts)
print("\nГенерация эмбеддингов для тестовой выборки...")
test_embeddings = get_embeddings(test_texts)

print(f"\nРазмерность эмбеддингов обучающей выборки: {train_embeddings.shape}")
print(f"Размерность эмбеддингов тестовой выборки: {test_embeddings.shape}")

train_embeddings = train_embeddings.astype(np.float32)
test_embeddings = test_embeddings.astype(np.float32)


Генерация эмбеддингов для обучающей выборки...


Generating Embeddings: 100%|██████████| 207/207 [06:04<00:00,  1.76s/it]



Генерация эмбеддингов для тестовой выборки...


Generating Embeddings: 100%|██████████| 14/14 [00:26<00:00,  1.92s/it]


Размерность эмбеддингов обучающей выборки: (13230, 1024)
Размерность эмбеддингов тестовой выборки: (883, 1024)


In [19]:
print("\nСоздание индекса nmslib...")
index = nmslib.init(method='hnsw', space='cosinesimil') # 'cosinesimil' - косинусное сходство, стандарт для эмбеддингов предложений.

index.addDataPointBatch(train_embeddings)

index_time_params = {'M': 20, 'efConstruction': 100, 'post': 0}
index.createIndex(index_time_params, print_progress=True)
print("Индекс построен.")


Создание индекса nmslib...
Индекс построен.


In [20]:
print("\nПоиск ближайших соседей для тестовой выборки...")
K = 1 

neighbors = index.knnQueryBatch(test_embeddings, k=K, num_threads=4) 
print("Поиск завершен.")


Поиск ближайших соседей для тестовой выборки...
Поиск завершен.


In [21]:
predicted_labels_k1 = []
for neighbor_info in neighbors:
    nearest_train_index = neighbor_info[0][0] 
    predicted_labels_k1.append(train_labels[nearest_train_index])

In [ ]:
accuracy_k1 = accuracy_score(test_labels, predicted_labels_k1)
print(f"\nТочность классификации (по 1 ближайшему соседу): {accuracy_k1:.4f}")

print("\nПоиск 5 ближайших соседей...")
K = 5
neighbors_k5 = index.knnQueryBatch(test_embeddings, k=K, num_threads=4)

predicted_labels_k5 = []
for neighbor_info in neighbors_k5:
    neighbor_indices = neighbor_info[0] # Индексы 5 ближайших соседей
    neighbor_actual_labels = [train_labels[i] for i in neighbor_indices] 
    most_common_label = Counter(neighbor_actual_labels).most_common(1)[0][0]
    predicted_labels_k5.append(most_common_label)

accuracy_k5 = accuracy_score(test_labels, predicted_labels_k5)
print(f"Точность классификации (k=5, голосование большинством): {accuracy_k5:.4f}")

print("\nГотово!")


Точность классификации (по 1 ближайшему соседу): 0.8777

Поиск 5 ближайших соседей...
Точность классификации (k=5, голосование большинством): 0.8652

Готово!
